In [62]:
from feature_functions import *

In [2]:
controlFiles, patientFiles = fileClassification()

In [48]:
'''
    Parameter: file path
    Returns: set of preprocessed data split into static and dynamic parts
'''

def data_preprocessing(filename):  
    df = pd.read_csv(filename, sep = ";", header = None)

    # split into static and dynamic dataframes
    static_df, dynamic_df = staticDynamicSplit(df)

    static_time = static_df[5]
    dynamic_time = dynamic_df[5]
    static_x = static_df[0]
    static_y = static_df[1]
    static_curve = np.array([static_x, static_y])

    dynamic_x = dynamic_df[0]
    dynamic_y = dynamic_df[1]
    dynamic_curve = np.array([dynamic_x, dynamic_y])

    static_velocity, static_acceleration, static_jerk, _, _ = smoothCurveFeature(static_curve, 1400, 10000)
    dynamic_velocity, dynamic_acceleration, dynamic_jerk, _, _ = smoothCurveFeature(dynamic_curve, 1400, 10000)

    _, _, _, static_curvature, _ = smoothCurveFeature(static_curve, 1400, 100000)
    _, _, _, dynamic_curvature, _ = smoothCurveFeature(dynamic_curve, 1400, 100000)
    
    static_pressure = static_df[3]
    static_risingIndex, static_fallingIndex = mainSignalThreshold(static_pressure)

    dynamic_pressure = dynamic_df[3]
    dynamic_risingIndex, dynamic_fallingIndex = mainSignalThreshold(dynamic_pressure)

    static_pressure_rising = static_pressure[0:static_risingIndex]
    static_pressure_main = static_pressure[static_risingIndex:static_fallingIndex]
    static_pressure_falling = static_pressure[static_fallingIndex:-1]

    dynamic_pressure_rising = dynamic_pressure[0:dynamic_risingIndex]
    dynamic_pressure_main = dynamic_pressure[dynamic_risingIndex:dynamic_fallingIndex]
    dynamic_pressure_falling = dynamic_pressure[dynamic_fallingIndex:-1]

    static_altitude = static_df[4]
    dynamic_altitude = dynamic_df[4]

    return static_x, static_y, dynamic_x, dynamic_y, static_velocity, static_acceleration, static_jerk, 
    static_curvature, static_pressure, static_pressure_rising, static_pressure_main, static_pressure_falling, 
    dynamic_velocity, dynamic_acceleration, dynamic_jerk, dynamic_curvature, dynamic_pressure,
    dynamic_pressure_rising, dynamic_pressure_main, dynamic_pressure_falling, static_altitude, dynamic_altitude, 
    time, static_risingIndex, static_fallingIndex, dynamic_risingIndex, dynamic_fallingIndex

In [31]:
'''
    Parameter: filename - file path of patient csv
    Returns: static features
'''

def static_calculate(filename): 
    # entropy: 
    static_x_entropy = entropyCalc(static_x)
    static_y_entropy = entropyCalc(static_y)
    
    # kinematic features
    # velocity features
    static_velocity_mean = np.mean(static_velocity)
    static_velocity_std = np.std(static_velocity)
    static_velocity_max = max(static_velocity)
    static_velocity_inversion_rate = rateOfInversions(static_velocity, static_time)

    # acceleration features
    static_acceleration_mean = np.mean(static_acceleration)
    static_acceleration_std = np.std(static_acceleration)
    static_acceleration_max = max(static_acceleration)
    static_acceleration_inversion_rate = rateOfInversions(static_acceleration, static_time)

    # jerk features
    static_jerk_mean = np.mean(static_jerk)
    static_jerk_std = np.std(static_jerk)
    static_jerk_max = max(static_jerk)
    static_jerk_inversion_rate = rateOfInversions(static_jerk, static_time)

    # curvature rate of inversion
    static_curv_inversion_rate = rateOfInversions(static_curvature, static_time)

    # fourier transform pressure
    static_pressure_low_freq, static_pressure_high_freq = fourierFreqCalc(static_pressure_main, static_time)
    
    # fourier transform altitude
    static_altitude_low_freq, static_altitude_high_freq = fourierFreqCalc(static_altitude, static_time)

    # pressure regression
    static_pressure_reg_main_r2, static_pressure_reg_main_x0, static_pressure_reg_main_x1, static_pressure_reg_main_sumresid = regression(static_pressure_main)

    # curvature regression
    static_curv_reg_r2, static_curv_reg_x0, static_curv_reg_x1, static_curv_reg_sumresid = regression(static_curvature)

    # velocity regression
    static_velocity_reg_r2, static_velocity_reg_x0, static_velocity_reg_x1, static_velocity_reg_sumresid = regression(static_velocity)

    # pressure rising & falling duration/range
    static_pressure_rising_duration = static_time[static_risingIndex] - static_time[0]
    static_pressure_rising_range = static_pressure[static_risingIndex] - static_pressure[0]
    static_pressure_falling_duration = static_time[len(static-time)-1] - static_time[static_fallingIndex]
    static_pressure_falling_range = static_pressure[static_fallingIndex] - static_pressure[len(static_time)-1]

    # skewness & kurtosis
    static_x_skewness = skew(static_x)
    static_x_kurtosis = kurtosis(static_x)
    static_y_skewness = skew(static_y)
    static_y_kurtosis = kurtosis(static_y)

    return static_x_entropy, static_y_entropy, static_velocity_mean, static_velocity_std, static_velocity_max, 
    static_velocity_inversion_rate, static_acceleration_mean, static_acceleration_std, static_acceleration_max, static_acceleration_inversion_rate, 
    static_jerk_mean, static_jerk_std, static_jerk_max, static_jerk_inversion_rate, static_curv_inversion_rate, static_pressure_low_freq, 
    static_pressure_high_freq, static_altitude_low_freq, static_altitude_high_freq, static_pressure_reg_main_r2, 
    static_pressure_reg_main_x0, static_pressure_reg_main_x1, static_pressure_reg_main_sumresid, 
    static_curv_reg_r2, static_curv_reg_x0, static_curv_reg_x1, static_curv_reg_sumresid, 
    static_velocity_reg_r2, static_velocity_reg_x0, static_velocity_reg_x1, static_velocity_reg_sumresid, 
    static_pressure_rising_duration, static_pressure_rising_range, static_pressure_falling_duration, static_pressure_falling_range, 
    static_x_skewness, static_x_kurtosis, static_y_skewness, static_y_kurtosis

In [33]:
'''
    Parameter: filename - file path of patient csv
    Returns: dynamic features
'''

def dynamic_calculate(filename): 
    # entropy: 
    dynamic_x_entropy = entropyCalc(dynamic_x)
    dynamic_y_entropy = entropyCalc(dynamic_y)

    # kinematic features
    # velocity features
    dynamic_velocity_mean = np.mean(dynamic_velocity)
    dynamic_velocity_std = np.std(dynamic_velocity)
    dynamic_velocity_max = max(dynamic_velocity)
    dynamic_velocity_inversion_rate = rateOfInversions(dynamic_velocity, dynamic_time)

    # acceleration features
    dynamic_acceleration_mean = np.mean(dynamic_acceleration)
    dynamic_acceleration_std = np.std(dynamic_acceleration)
    dynamic_acceleration_max = max(dynamic_acceleration)
    dynamic_acceleration_inversion_rate = rateOfInversions(dynamic_acceleration, dynamic_time)

    # jerk features
    dynamic_jerk_mean = np.mean(dynamic_jerk)
    dynamic_jerk_std = np.std(dynamic_jerk)
    dynamic_jerk_max = max(dynamic_jerk)
    dynamic_jerk_inversion_rate = rateOfInversions(dynamic_jerk, dynamic_time)

    # curvature rate of inversion
    dynamic_curv_inversion_rate = rateOfInversions(dynamic_curvature, dynamic_time)

    # pressure rising & falling duration/range
    dynamic_pressure_rising_duration = dynamic_time[dynamic_risingIndex] - dynamic_time[0]
    dynamic_pressure_rising_range = dynamic_pressure[dynamic_risingIndex] - dynamic_pressure[0]
    dynamic_pressure_falling_duration = dynamic_time[len(dynamic_time)-1] - dynamic_time[dynamic_fallingIndex]
    dynamic_pressure_falling_range = dynamic_pressure[dynamic_fallingIndex] - dynamic_pressure[len(dynamic_time)-1]

    # skewness & kurtosis
    dynamic_x_skewness = skew(dynamic_x)
    dynamic_y_skewness = skew(dynamic_y)
    dynamic_x_kurtosis = kurtosis(dynamic_x)
    dynamic_y_kurtosis = kurtosis(dynamic_y)

    # fourier transform pressure
    dynamic_pressure_low_freq, dynamic_pressure_high_freq = fourierFreqCalc(dynamic_pressure_main, dynamic_time)
    
    # fourier transform altitude
    dynamic_altitude_low_freq, dynamic_altitude_high_freq = fourierFreqCalc(dynamic_altitude, dynamic_time)

    # pressure regression
    dynamic_pressure_reg_main_r2, dynamic_pressure_reg_main_x0, dynamic_pressure_reg_main_x1, dynamic_pressure_reg_main_sumresid = regression(dynamic_pressure_main)

    # curvature regression
    dynamic_curv_reg_r2, dynamic_curv_reg_x0, dynamic_curv_reg_x1, dynamic_curv_reg_sumresid = regression(dynamic_curvature)

    # velocity regression
    dynamic_velocity_reg_r2, dynamic_velocity_reg_x0, dynamic_velocity_reg_x1, dynamic_velocity_reg_sumresid = regression(dynamic_velocity)

    return dynamic_x_entropy, dynamic_y_entropy, dynamic_velocity_mean, dynamic_velocity_std, dynamic_velocity_max, 
    dynamic_velocity_inversion_rate, dynamic_acceleration_mean, dynamic_acceleration_std, dynamic_acceleration_max, 
    dynamic_acceleration_inversion_rate, dynamic_jerk_mean, dynamic_jerk_std, dynamic_jerk_max, dynamic_jerk_inversion_rate, 
    dynamic_curv_inversion_rate, dynamic_pressure_low_freq, dynamic_pressure_high_freq, dynamic_altitude_low_freq, dynamic_altitude_high_freq, 
    dynamic_altitude_low_freq, dynamic_altitude_high_freq, 
    dynamic_pressure_rising_duration, dynamic_pressure_rising_range, dynamic_pressure_falling_duration, 
    dynamic_pressure_falling_range, dynamic_x_skewness, dynamic_y_skewness, dynamic_x_kurtosis, dynamic_y_kurtosis,
    dynamic_pressure_reg_main_r2, dynamic_pressure_reg_main_x0, dynamic_pressure_reg_main_x1, 
    dynamic_pressure_reg_main_sumresid, dynamic_curv_reg_r2, dynamic_curv_reg_x0, dynamic_curv_reg_x1, dynamic_curv_reg_sumresid, 
    dynamic_velocity_reg_r2, dynamic_velocity_reg_x0, dynamic_velocity_reg_x1, dynamic_velocity_reg_sumresid

In [65]:
# CALCULATE ALL STATIC FEATURES AND PUT INTO DATAFRAME

static_subject_id = []
static_category = []
static_mean_vel = []
static_max_vel = []
static_vel_std = []
static_vel_niv = [] #rate
static_nvv = [] 
static_mean_accel = []
static_max_accel = []
static_std_accel = []
static_nia = [] #rate
static_mean_jerk = []  
static_max_jerk = []
static_std_jerk = []
static_nij = [] #rate
static_duration = []
static_nic = []
static_mean_pressure = []
static_max_pressure = []
static_std_pressure = []
static_rate_ip = []
static_x_entropy = []
static_y_entropy = []
static_x_skewness = []
static_y_skewness = []
static_x_kurtosis = []
static_y_kurtosis = []
static_pressure_high_freq = []
static_pressure_low_freq = []
static_altitude_high_freq = []
static_altitude_low_freq = []
static_pressure_reg_r2 = []
static_pressure_reg_x0 = []
static_pressure_reg_x1 = []
static_pressure_reg_sumreside = []
static_curv_reg_r2 = []
static_curv_reg_x0 = []
static_curv_reg_x1 = []
static_curv_reg_sumreside = []
static_vel_reg_r2 = []
static_vel_reg_x0 = []
static_vel_reg_x1 = []
static_vel_reg_sumreside = []

df = pd.DataFrame({'Static Subject ID' : static_subject_id,
'Mean Velocity': static_mean_vel, 'Velocity Std': static_vel_std, 'Max Velocity': static_max_vel, 'NVV': static_nvv, 'NIV': static_vel_niv,
'Mean Acceleration': static_mean_accel, 'Accleration Std': static_std_accel, 'Max Acceleration': static_max_accel, 'NIA': static_nia,
'Mean Jerk': static_mean_jerk, 'Jerk Std': static_std_jerk, 'Max Jerk': static_max_jerk, 'NIJ': static_nij, 
'Duration': static_duration, 'NIC': static_nic,
'Mean Pressure': static_mean_pressure, 'Max Pressure': static_max_pressure, 'Pressure Std': static_std_pressure, 'Rate of Inversion in Pressure': static_rate_ip,
'X Entropy': static_x_entropy, 'Y Entropy': static_y_entropy, 
'X Skewness': static_x_skewness, 'Y Skewness': static_y_skewness, 
'X Kurtosis': static_x_kurtosis, 'Y Kurtosis': static_y_kurtosis,
'Pressure High Freq': static_pressure_high_freq, 'Pressure Low Freq': static_pressure_low_freq, 
'Altitude High Freq': static_altitude_high_freq, 'Altitude Low Freq': static_altitude_low_freq, 
'Pressure Reg R^2': static_pressure_reg_r2, 'Pressure Reg x0': static_pressure_reg_x0, 'Pressure Reg x1': static_pressure_reg_x1, 'Pressure Reg SumResids': static_pressure_reg_sumreside,
'Curv Reg R^2': static_curv_reg_r2, 'Curv Reg x0': static_curv_reg_x0, 'Curv Reg x1': static_curv_reg_x1, 'Curv Reg SumResids': static_curv_reg_sumreside,
'Vel Reg R^2': static_vel_reg_r2, 'Vel Reg x0': static_vel_reg_x0, 'Vel Reg x1': static_vel_reg_x1, 'Vel Reg SumResids': static_vel_reg_sumreside})

for filename in controlFiles:
    # retrieve all preprocessed data
    (static_x, static_y, dynamic_x, dynamic_y, static_velocity, static_acceleration, static_jerk, 
    static_curvature, static_pressure, static_pressure_rising, static_pressure_main, static_pressure_falling, 
    dynamic_velocity, dynamic_acceleration, dynamic_jerk, dynamic_curvature, dynamic_pressure,
    dynamic_pressure_rising, dynamic_pressure_main, dynamic_pressure_falling, static_altitude, dynamic_altitude, 
    time, static_risingIndex, static_fallingIndex, dynamic_risingIndex, dynamic_fallingIndex) = data_preprocessing(filename) 
    
    a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,a1,b1,c1,d1,e1,f1,g1,h1,i1,j1,k1,l1,m1,n1,o1,p1 = static_calculate(filename)

    static_category.append(0)
    static_subject_id.append(a)
    static_mean_vel.append(b)
    static_max_vel.append(c)
    static_vel_std.append(d)
    static_vel_niv.append(e) #rate
    static_nvv.append(f)
    static_mean_accel.append(g)
    static_max_accel.append(h)
    static_std_accel.append(i)
    static_nia.append(j) #rate
    static_mean_jerk.append(k)
    static_max_jerk.append(l)
    static_std_jerk.append(m)
    static_nij.append(n) #rate
    static_duration.append(o)
    static_nic.append(p)
    static_mean_pressure.append(q)
    static_max_pressure.append(r)
    static_std_pressure.append(s)
    static_rate_ip.append(t)
    static_x_entropy.append(u)
    static_y_entropy.append(v)
    static_x_skewness.append(w)
    static_y_skewness.append(x)
    static_x_kurtosis.append(y)
    static_y_kurtosis.append(z)
    static_pressure_high_freq.append(a1)
    static_pressure_low_freq.append(b1)
    static_altitude_high_freq.append(c1)
    static_altitude_low_freq.append(d1)
    static_pressure_reg_r2.append(e1)
    static_pressure_reg_x0.append(f1)
    static_pressure_reg_x1.append(g1)
    static_pressure_reg_sumreside.append(h1)
    static_curv_reg_r2.append(i1)
    static_curv_reg_x0.append(j1)
    static_curv_reg_x1.append(k1)
    static_curv_reg_sumreside.append(l1)
    static_vel_reg_r2.append(m1)
    static_vel_reg_x0.append(n1)
    static_vel_reg_x1.append(o1)
    static_vel_reg_sumreside.append(p1)

    
for filename in patientFiles:
    # retrieve all preprocessed data
    static_x, static_y, dynamic_x, dynamic_y, static_velocity, static_acceleration, static_jerk, 
    static_curvature, static_pressure, static_pressure_rising, static_pressure_main, static_pressure_falling, 
    dynamic_velocity, dynamic_acceleration, dynamic_jerk, dynamic_curvature, dynamic_pressure,
    dynamic_pressure_rising, dynamic_pressure_main, dynamic_pressure_falling, static_altitude, dynamic_altitude, 
    time, static_risingIndex, static_fallingIndex, dynamic_risingIndex, dynamic_fallingIndex = data_preprocessing(filename)
    
    a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,a1,b1,c1,d1,e1,f1, g1 = static_calculate(filename)

    static_category.append(1)
    static_subject_id.append(a)
    static_mean_vel.append(b)
    static_max_vel.append(c)
    static_vel_std.append(d)
    static_vel_niv.append(e) #rate
    static_nvv.append(f)
    static_mean_accel.append(g)
    static_max_accel.append(h)
    static_std_accel.append(i)
    static_nia.append(j) #rate
    static_mean_jerk.append(k)
    static_max_jerk.append(l)
    static_std_jerk.append(m)
    static_nij.append(n) #rate
    static_duration.append(o)
    static_nic.append(p)
    static_mean_pressure.append(q)
    static_max_pressure.append(r)
    static_std_pressure.append(s)
    static_rate_ip.append(t)
    static_x_entropy.append(u)
    static_y_entropy.append(v)
    static_x_skewness.append(w)
    static_y_skewness.append(x)
    static_x_kurtosis.append(y)
    static_y_kurtosis.append(z)
    static_pressure_high_freq.append(a1)
    static_pressure_low_freq.append(b1)
    static_altitude_high_freq.append(c1)
    static_altitude_low_freq.append(d1)
    static_pressure_reg_r2.append(e1)
    static_pressure_reg_x0.append(f1)
    static_pressure_reg_x1.append(g1)
    static_pressure_reg_sumreside.append(h1)
    static_curv_reg_r2.append(i1)
    static_curv_reg_x0.append(j1)
    static_curv_reg_x1.append(k1)
    static_curv_reg_sumreside.append(l1)
    static_vel_reg_r2.append(m1)
    static_vel_reg_x0.append(n1)
    static_vel_reg_x1.append(o1)
    static_vel_reg_sumreside.append(p1)

NameError: name 'interpolate' is not defined

In [ ]:
# CALCULATE ALL DYNAMIC FEATURES AND PUT INTO DATAFRAME


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=54ea7590-1f30-42ee-b9c1-c929b94e3f2f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>